#### Import Libraries

In [1]:
import os, sys
import yfinance as yf
# import ta
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import date, timedelta, datetime
from itertools import product
from IPython.display import clear_output
import matplotlib.pyplot as plt
from csv import writer
import warnings
import talib as ta
import math
import re
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

/home/diti/miniconda3/envs/devbox/lib/python3.12/site-packages/telegram/utils/request.py:49: UserWarning: python-telegram-bot is using upstream urllib3. This is allowed but not supported by python-telegram-bot maintainers.
  warnings.warn(


In [77]:
def print_exception_detail(err, *args, **kwargs):
    exc_type, exc_obj, exc_tb = sys.exc_info()
    if len(args):
        for _arg in args:
            print(_arg)
    print(f"Error Message: {err}")
    print(f"Error at line number: {exc_tb.tb_lineno}")

##### Tickers of S&P100 and Treasury Bond Rates of 1day frequency

In [57]:
treasury_bond = ["^TYX"]
ticker = ["NVDA","AAPL","MSFT","AMZN","GOOGL","GOOG","META","TSLA","AVGO","COST","NFLX","ASML","TMUS","AMD","CSCO","PEP","ADBE","LIN","AZN","TXN","QCOM	","INTU","ISRG","AMGN	","CMCSA","PDD","BKNG","AMAT","ARM","HON","VRTX","PANW","ADP","MU","GILD","ADI","SBUX","MELI","INTC","LRCX","KLAC","MDLZ","ABNB","REGN","CTAS","SNPS","CDNS","PYPL","CRWD","MRVL","MAR","CEG","CSX","ORLY","DASH","WDAY","CHTR","ADSK","FTNT","TTD","ROP","PCAR","NXPI","TEAM","FANG","MNST","CPRT","PAYX","AEP","ODFL","ROST","FAST","KDP","DDOG","EA","BKR","KHC","MCHP","VRSK","GEHC","CTSH","LULU","EXC","XEL","CCEP","IDXX","ON","CSGP","ZS","TTWO","ANSS","CDW","DXCM","BIIB","GFS","ILMN","MDB","WBD","MRNA","DLTR","WBA"]
ticker = [_t.strip() for _t in ticker]

##### Methods to download data

In [32]:
def download_data(ticker, interval='1d'):
    # dt_range = pd.date_range('2019-12-31', '2020-12-31', freq='6m') 
    # define start and end date
    # end_date = datetime.now()-timedelta(days=30)
    # start_date = end_date - timedelta(days=390)

    end_date = datetime.now() #- timedelta(days=30)
    start_date = end_date - timedelta(days=1855)

    # extract data from yahoo finanace
    btc_price = vbt.YFData.download(
    ticker,
    interval=interval,
    start = start_date,
    end = end_date,
    missing_index='drop').get("Close")
    df = pd.DataFrame(btc_price)
    df.rename(columns={"Close":ticker}, inplace=True)
    df.index=df.index.strftime("%Y-%m-%d")
    df.index = pd.to_datetime(df.index)
    return df

In [58]:
df = pd.DataFrame()
for _t in ticker:
    try:
        if df.empty:
            df=download_data(_t)
        else:
            df1 = pd.DataFrame()
            df1 = download_data(_t)
            # per_chg = len(df.index.difference(df2.index)) #/len(df.index))*100
            # print(per_chg)
            # if per_chg<518:
            df = pd.merge(df, df1, left_index=True, right_index=True, how='inner')  # Only keep matching rows
    except Exception as err:
        print_exception_detail(err)

In [70]:
df_treasury = download_data(treasury_bond[0])

In [79]:
df_treasury[f"{treasury_bond[0]}_pct_change_20"] = df_treasury[f"{treasury_bond[0]}"].pct_change(20)

In [59]:
df.shape

(291, 101)

In [71]:
df_treasury.head()

,^TYX
Date,
2019-10-11,2.215
2019-10-14,2.197
2019-10-15,2.233
2019-10-16,2.233
2019-10-17,2.243


In [80]:
df = pd.merge(df, df_treasury, left_index=True, right_index=True, how="inner")

In [73]:
df.shape

(291, 102)

##### Retain the original dataframe and prepare three different dataframe
1. df_tech : Dataframe to evaluate technical scores for last n days (in our case n=20 days)
2. df_sharpe : Dataframe to evaluate the sharpe ratio for m days (in our case m>=250 days)
3. df_performance : Dataframe to evaluate the performance rank for l days (in our case l=7 days)
4. df_slope : Dataframe to evaluate slope for k days (in our case k=Average of price of 6,7 days)

##### Percentile Method

$
 \frac{rank of an element}{total number of element}\,* 100 
$

In [60]:
def percentile(window):
    index = window.index[-1]
    window = window.rank(method="min", ascending=True).apply(lambda x: (x/20)*100)
    return window.loc[index]

### Indicator

##### There are two sets of indicators used
1. ##### For timing the entry and exit; RSI, Stochastic and Bollingier
2. ##### For selection of best stocks; Performance Rank and Sharpe Ratio
3. ##### As an additional support to our decision; Sentiment of S&P and Slope of the assets for last 1 or 2 weeks

#### Stochastic Indicator

Stochastic indicator is a momentum indicator that uses support and resistance levels as an oscillator

%K = (Current Close - Lowest Low)/(Highest High - Lowest Low) * 100
%D = 3-day SMA of %K

We will focus on the stock with %D-Slow turnaround in the bottom as they will signal an entry. 
For normalization of the historic %D, the percentile of value wrt last 20 days is evaluated.

#### RSI Indicator


#### Bollingier

In [64]:
def stochastic_k(price, period):
    index = price.index[-1]
    sorted_price = price.sort_values(ascending=True)
    closing_price = price.loc[index]
    highest_high = sorted_price.iloc[-1]
    lowest_low = sorted_price.iloc[0]
    per_k = (closing_price - lowest_low)/(highest_high-lowest_low)*100
    return per_k

In [65]:
# To evaluate zscore using a penalized version error function
def zscore(x):
    return (math.erf(6-(12*(x/100))))

In [82]:
def rank_asset(row):
    size = len(row)
    sorted_row = row.sort_values(ascending=True) 
    buy_recom = [{col[:col.find('_')]:sorted_row[col]} for col in list(sorted_row[math.floor(size*.75):].index)]
    sell_recom = [{col[:col.find('_')]:sorted_row[col]} for col in list(sorted_row[:math.floor(size*.25)].index)]
    return buy_recom, sell_recom

In [129]:
for _t in ticker:
    try:
        # print(_t)
        # # Technical Indicator
        # # -------------------
        
        # # RSI of last n days
        # df[f"{_t}_rsi_20"] = ta.RSI(df[_t], timeperiod=20)
        # # Percentile of RSI value wrt last n days. This will give us the respective position
        # df[f"{_t}_rsi_20_percentile_last_20"]=df[f"{_t}_rsi_20"].rolling(window=20).apply(percentile)

        # # Stochastic of last n days
        # df[f"{_t}_sto_fast_20"]=df[f"{_t}"].rolling(window=20).apply(lambda x: stochastic_k(x, 20))
        # df[f"{_t}_sto_slow_20"]=ta.SMA(df[f"{_t}_sto_fast_20"],3)
        # # Percentile of Stochastic slow value wrt last n days. This will give us the respective position
        # df[f"{_t}_sto_slow_20_percentile_last_20"]=df[f"{_t}_sto_slow_20"].rolling(window=20).apply(percentile)

        # # Bollingier Indicator for last n days
        # df[f"{_t}_sma_20"] = ta.SMA(df[f"{_t}"],20)
        # df[f"{_t}_std_20"] = df[f"{_t}"].rolling(window=20).std()
        # # Lamda = (P(t) - SMA_20)/Std_20
        # df[f"{_t}_lamda_20"] = (df[f"{_t}"] - df[f"{_t}_sma_20"])/df[f"{_t}_std_20"]
        # # Percentile of lambda value wrt last n days. This will give us the respective position
        # df[f"{_t}_lamda_20_percentile_lst_20"] = df[f"{_t}_lamda_20"].rolling(window=20).apply(percentile)

        # # z- scores
        # # ---------

        # # z-score of RSI
        # df[f"{_t}_rsi_20_zscore"] = df[f"{_t}_rsi_20_percentile_last_20"].apply(zscore)
        # # z-score of Stochastic Slow
        # df[f"{_t}_sto_slow_20_zscore"] = df[f"{_t}_sto_slow_20_percentile_last_20"].apply(zscore)
        # # z-score of Lambda - a derived bollinger value
        # df[f"{_t}_lamda_20_zscore"] = df[f"{_t}_lamda_20_percentile_lst_20"].apply(zscore)

        # # Mean of z-scores of RSI, Stochastic slow and Lambda
        # # z-score tends to 1 means indication of reversal at the bottom 
        # df[f"{_t}_mean_zscore"] = (df[f"{_t}_rsi_20_zscore"] + df[f"{_t}_sto_slow_20_zscore"] + df[f"{_t}_lamda_20_zscore"])/3

        # # Selection Indicators
        # # --------------------

        # # Sharpe Ratio - This will be evaluated for long period
        # df[f"{_t}_pct_change_20"] = df[f"{_t}"].pct_change(20)
        # df[f"{_t}_return_std_20"] = df[f"{_t}_pct_change_20"].rolling(window=20).std()
        # df[f"{_t}_sharpe_ratio"] = (df[f"{_t}_pct_change_20"] - df[f"{treasury_bond[0]}_pct_change_20"])/df[f"{_t}_return_std_20"]

        # # Performance Rank - This will be evaluated for last 7 days
        # df[f"{_t}_performance_rank"] = ((df[f"{_t}"] - df[f"{_t}"].shift(7))/df[f"{_t}"])*100

        # # Slope of stock at each timestamp; evaluated {[(P(t)+P(t-1))-(P(t-7)+P(t-8))]/2*P(t)}
        # df[f"{_t}_shift1"] = df[f"{_t}"].shift(1)
        # df[f"{_t}_shift6"] = df[f"{_t}"].shift(6)
        # df[f"{_t}_shift7"] = df[f"{_t}"].shift(7)
        # df[f"{_t}_slope"] = ((df[f"{_t}"] + df[f"{_t}_shift1"])-(df[f"{_t}_shift6"] + df[f"{_t}_shift7"]))/(2*df[f"{_t}"] )

        # Create three separate set : Top quartile set for buy and low quartile set for sell

        # Set of stocks on mean z-score
        df[["buy_recom_zscore", "sell_recom_zscore"]] = \
                df[[col for col in df.columns if re.search(".*mean_zscore$", col)]].apply(rank_asset, axis=1, \
                        result_type='expand')
        
        # # Set of stocks on the sharpe ratio
        # df[["buy_recom_sharpe", "sell_recom_sharpe"]] = \
        #         df[[col for col in df.columns if re.search(".*sharpe_ratio$", col)]].apply(rank_asset, axis=1, result_type='expand')
        
        # # Set of stocks on the sharpe ratio
        # df[["buy_recom_per", "sell_recom_per"]] = \
        #         df[[col for col in df.columns if re.search(".*performance_rank$", col)]].apply(rank_asset, axis=1, result_type='expand')
    except Exception as err:
        print_exception_detail(err)
        sys.exit()
    

In [130]:
df.to_csv("out.csv")

In [131]:
df.to_csv("out_en_utf.csv", encoding='utf-8', index=False, header=True)

In [132]:
df.tail()

,NVDA,AAPL,MSFT,AMZN,GOOGL,GOOG,META,TSLA,AVGO,COST,...,DLTR_shift7,DLTR_slope,WBA_sharpe_ratio,WBA_performance_rank,WBA_shift1,WBA_shift6,WBA_shift7,WBA_slope,buy_recom_zscore,sell_recom_zscore
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-01,135.399994,222.910004,410.369995,197.929993,171.289993,172.649994,567.159973,248.979996,168.919998,877.309998,...,66.129997,-0.013814,0.207415,1.162787,9.46,9.34,9.35,0.012156,"[{'CRWD': 0.9999999990443772}, {'FTNT': 0.9999...","[{'MRNA': -0.9999999999999926}, {'WBD': -0.999..."
2024-11-04,136.050003,222.009995,408.459991,195.779999,169.240005,170.679993,560.679993,242.839996,168.550003,886.070007,...,66.949997,0.000075,0.105502,-0.107184,9.46,9.26,9.34,0.010182,"[{'PANW': 0.9999998813789874}, {'KHC': 0.99999...","[{'DLTR': -0.9999999999999926}, {'CHTR': -0.99..."
2024-11-05,139.910004,223.449997,411.459991,199.500000,169.740005,171.410004,572.429993,251.440002,173.899994,890.169983,...,66.449997,0.006515,0.387949,3.742200,9.33,9.44,9.26,0.012994,"[{'PEP': 0.9995408721145723}, {'FANG': 0.99977...","[{'DLTR': -0.9999999999999926}, {'AMZN': -0.99..."
2024-11-06,145.610001,222.720001,420.179993,207.089996,176.509995,178.330002,572.049988,288.529999,179.550003,899.250000,...,66.260002,-0.012658,-0.326911,-2.275190,9.62,9.35,9.44,0.003250,"[{'LULU': 0.3333333323815019}, {'ASML': 0.3335...","[{'ODFL': -1.0}, {'BKR': -1.0}, {'COST': -1.0}..."
2024-11-07,148.880005,227.479996,425.429993,210.050003,180.750000,182.279999,591.700012,296.910004,183.809998,913.929993,...,64.500000,-0.014636,-0.122428,-0.862998,9.23,9.24,9.35,-0.004854,"[{'REGN': -0.3331038287011183}, {'ROST': -0.32...","[{'WBD': -1.0}, {'INTC': -1.0}, {'WDAY': -1.0}..."


In [304]:
# df_sharpe = df_asset[[col for col in df_asset.columns for _t in ticker if re.search(f"{_t}.*pct_change_20$|^{_t}$", col)]]

In [306]:
# df_sharpe = pd.merge(df_sharpe, df_treasury, left_index=True, right_index=True, how="inner")

In [317]:
# ticker = [col for col in df_sharpe.columns for _t in ticker if re.search(f"{_t}.*pct_change_20$", col)]

In [324]:
# df_sharpe.drop(ticker+treasury_bond, axis=1, inplace=True)

In [336]:
# for _t in ticker:
#     df_sharpe[f"{_t}_return_std_20"] = df_sharpe[f"{_t}_pct_change_20"].rolling(window=20).std()
#     df_sharpe[f"{_t}_sharpe_ratio"] = (df_sharpe[f"{_t}_pct_change_20"] - df_sharpe[f"{treasury_bond[0]}_pct_change_20"])/df_sharpe[f"{_t}_return_std_20"]

In [365]:
# import sys
# def rank_asset_sharpe(row):
#     # print(row.sort_values(ascending=True))
#     size = len(row)
#     sorted_row = row.sort_values(ascending=True) 
#     buy_recom_tech = [{col[:col.find('_')]:sorted_row[col]} for col in list(sorted_row[math.floor(size*.75):].index)]
#     sell_recom_tech = [{col[:col.find('_')]:sorted_row[col]} for col in list(sorted_row[:math.floor(size*.25)].index)]
#     return buy_recom_tech, sell_recom_tech

In [357]:
# df_sharpe.dropna(inplace=True)

In [366]:
# # Apply the function row-wise and unpack the results into multiple columns
# df_sharpe[["buy_recommendation_sharpe", "sell_recommendation_sharpe"]] = \
# df_sharpe[[col for col in df_sharpe.columns if re.search(f".*sharpe_ratio$", col)]].apply(rank_asset_sharpe, axis=1, result_type='expand')

In [266]:
df_original = df.copy()

In [289]:
df_asset = df_original.copy()
df_treasury = df.copy()

In [371]:
df_performance_rank = df_original.copy()

In [374]:
df_performance_rank = df_performance_rank[ticker]

In [375]:
# # for _t in ticker:
#     df_performance_rank[f"{_t}_performance_rank"] = ((df_performance_rank[f"{_t}"] - df_performance_rank[f"{_t}"].shift(5))/df_performance_rank[f"{_t}"])*100

In [379]:
# import sys
# def rank_asset_performance(row):
#     # print(row.sort_values(ascending=True))
#     size = len(row)
#     sorted_row = row.sort_values(ascending=True) 
#     buy_recom_per = [{col[:col.find('_')]:sorted_row[col]} for col in list(sorted_row[math.floor(size*.75):].index)]
#     sell_recom_per = [{col[:col.find('_')]:sorted_row[col]} for col in list(sorted_row[:math.floor(size*.25)].index)]
#     return buy_recom_per, sell_recom_per

In [380]:
# # Apply the function row-wise and unpack the results into multiple columns
# df_performance_rank[["buy_recommendation_per", "sell_recommendation_per"]] = df_performance_rank.apply(rank_asset_performance, axis=1, result_type='expand')

In [383]:
# df_performance_rank.dropna(inplace=True)

In [146]:
#testing of percentile of last 20 days rsi
# a = df[21:41]["MSFT_rsi_20"].sort_values(ascending=True)
# a
# pd.to_datetime(a.index)
# a=pd.DataFrame(a)
# a["rank"] = a["MSFT_rsi_20"].rank(method="min", ascending=True)
# a["percentile"] = a["rank"].apply(lambda x: (x/20)*100)

In [268]:
# df.dropna(inplace=True)

In [270]:
# list_of_cols = [col for col in df.columns for _t in ticker if re.search(f"{_t}.*zscore$|^{_t}$", col)]

In [271]:
# df = df[list_of_cols]

In [272]:
# for _t in ticker:
#     try:
#         df[f"{_t}_cum_zscore"] = (df[f"{_t}_rsi_20_zscore"] + df[f"{_t}_sto_slow_20_zscore"] + df[f"{_t}_lamda_20_zscore"])/3
#     except Exception as err:
#         print(f"Error : {err}")

In [273]:
# df_cum_score = df[[col for col in df.columns for _t in ticker if re.search(f"{_t}.*cum_zscore$", col)]]

In [369]:
# import sys
# def rank_asset(row):
#     # print(row.sort_values(ascending=True))
#     size = len(row)
#     sorted_row = row.sort_values(ascending=True) 
#     buy_recom_tech = [{col[:col.find('_')]:sorted_row[col]} for col in list(sorted_row[math.floor(size*.75):].index)]
#     sell_recom_tech = [{col[:col.find('_')]:sorted_row[col]} for col in list(sorted_row[:math.floor(size*.25)].index)]
#     return buy_recom_tech, sell_recom_tech

In [394]:
# df_cum_score.drop(columns=['buy_recommendation', 'sell_recommendation'], axis=1, inplace=True)

In [396]:

# # Apply the function row-wise and unpack the results into multiple columns
# df_cum_score[["buy_recommendation_zscore", "sell_recommendation_zscore"]] = df_cum_score.apply(rank_asset, axis=1, result_type='expand')

In [397]:
# df = df_cum_score.merge(df_sharpe, left_index=True, right_index=True).merge(df_performance_rank, left_index=True, right_index=True)

In [133]:
df_recom = df[[col for col in df.columns if re.search(".*buy_recom.*", col)]]

In [134]:
df_recom.dropna(inplace=True)

/tmp/ipykernel_591139/601615114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recom.dropna(inplace=True)


In [135]:
df_recom = df_recom.iloc[-60:]

In [136]:
s1 = set([list(_d.keys())[0] for _d in df_recom.loc["2024-08-15"]["buy_recom_sharpe"]])

In [119]:
s2 = set([list(_d.keys())[0] for _d in df_recom.loc["2024-08-15"]["buy_recom_per"]])

In [138]:
s3 = set([list(_d.keys())[0] for _d in df_recom.loc["2024-08-15"]["buy_recom_zscore"]])

In [120]:
s1.intersection(s2)

{'FTNT', 'ILMN', 'MELI', 'PDD', 'REGN', 'SBUX'}

In [139]:
s3.intersection(s1) #.intersection(s2)

{'ADP', 'AEP', 'CTAS', 'EXC', 'ORLY', 'TMUS', 'XEL'}

In [141]:
s3.intersection(s2)

set()

In [145]:
df[[col for col in df.columns if re.search(".*slope.*", col)]].loc["2024-08-15"]

NVDA_slope     0.153752
AAPL_slope     0.066465
MSFT_slope     0.048139
AMZN_slope     0.064728
GOOGL_slope    0.013763
                 ...   
MDB_slope      0.089079
WBD_slope     -0.074586
MRNA_slope     0.028977
DLTR_slope    -0.010546
WBA_slope     -0.020333
Name: 2024-08-15 00:00:00, Length: 101, dtype: float64

In [137]:
df_recom.loc["2024-08-15"]["buy_recom_zscore"]

[{'VRSK': -0.4025707272319509},
 {'TMUS': -0.36321394692545583},
 {'CTAS': -0.3333333323815019},
 {'BIIB': -0.333325969834334},
 {'ADP': -0.3331038287011183},
 {'CMCSA': -0.30343799274321176},
 {'CTSH': -0.23118058558509938},
 {'MRNA': -0.20128536361597568},
 {'TEAM': -0.1284114702626017},
 {'CSGP': 0.20128536361597524},
 {'EXC': 0.3333333323815019},
 {'XEL': 0.6367639625775455},
 {'VRTX': 0.8344201902377639},
 {'FAST': 0.8378271850603053},
 {'ODFL': 0.8380493261935208},
 {'AMGN': 0.8606790313731291},
 {'BKR': 0.8643154122068878},
 {'WBD': 0.9696456501454485},
 {'ORLY': 0.9700972959070948},
 {'CSX': 0.9927270010867031},
 {'MNST': 0.996126632414029},
 {'PEP': 0.9963634995896281},
 {'KHC': 0.9963635005414521},
 {'GILD': 0.9999852720501695},
 {'CHTR': 0.9999999990443772},
 {'AEP': 0.9999999999999777}]